In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Carga de Datos

In [2]:
postulantesEducacion = pd.read_csv('resources/tar/fiuba_1_postulantes_educacion.csv')
postulantesGenero = pd.read_csv('resources/tar/fiuba_2_postulantes_genero_y_edad.csv')
vistas = pd.read_csv('resources/tar/fiuba_3_vistas.csv')
postulaciones = pd.read_csv('resources/tar/fiuba_4_postulaciones.csv')
avisosOnline = pd.read_csv('resources/tar/fiuba_5_avisos_online.csv')
avisosDetalle = pd.read_csv('resources/tar/fiuba_6_avisos_detalle.csv')


#SEGUNDOS ARCHIVOS
postulantesEducacion2 = pd.read_csv('resources/datos_navent_fiuba/fiuba_1_postulantes_educacion.csv')
postulantesGenero2 = pd.read_csv('resources/datos_navent_fiuba/fiuba_2_postulantes_genero_y_edad.csv')
vistas2 = pd.read_csv('resources/datos_navent_fiuba/fiuba_3_vistas.csv')
postulaciones2 = pd.read_csv('resources/datos_navent_fiuba/fiuba_4_postulaciones.csv')
avisosOnline2 = pd.read_csv('resources/datos_navent_fiuba/fiuba_5_avisos_online.csv')
avisosDetalle2 = pd.read_csv('resources/datos_navent_fiuba/fiuba_6_avisos_detalle.csv')

#cargamos zips
avisosDetalleZIP = pd.read_csv('resources/zip/fiuba_6_avisos_detalle.csv')
avisosDetalleMissing = pd.read_csv('resources/zip/fiuba_6_avisos_missing.csv')

postulantesEducacionZip = pd.read_csv('resources/zip/fiuba_1_postulantes_educacion.csv')
postulantesGeneroZip = pd.read_csv('resources/zip/fiuba_2_postulantes_genero_y_edad.csv')

vistasZIP = pd.read_csv('resources/zip/fiuba_3_vistas.csv')

In [4]:
#Set de test
testingSet = pd.read_csv('resources/test_final_100k.csv')

### Unimos datos de archivos

In [9]:
#Vistas
vistasCompletas = pd.concat([vistas,vistas2])
vistasCompletas['idaviso'] = vistasCompletas['idAviso']
vistasCompletas.drop('idAviso',inplace=True,axis=1)

#Genero y educacion
educacionTodas = pd.concat([postulantesEducacion,postulantesEducacionZip,postulantesEducacion2])
generoTodas = pd.concat([postulantesGenero,postulantesGeneroZip,postulantesGenero2])

#Avisos Detalle
detalleCompletas = pd.concat([avisosDetalle,avisosDetalle2])

#Postulacion
postulaciones = pd.concat([postulaciones,postulaciones2])


##Unimos lo de los zips
detalleCompletasZIP = pd.concat([avisosDetalleZIP,avisosDetalleMissing])

### Genero avisos no postulados

In [131]:
#vf = vistasCompletas[['idaviso','idpostulante']]
#vf= vf.drop_duplicates()
#len(vf)

#postulacionesConVistas = pd.merge(vf,postulaciones,on=['idaviso','idpostulante'],how='outer')
#avisosNoPostulado = postulacionesConVistas[postulacionesConVistas['fechapostulacion'].isnull()]
#avisosNoPostulado.drop('fechapostulacion',axis=1,inplace=True)

In [95]:
len(avisosNoPostulado)

1870115

In [12]:
noPostu = pd.read_csv('nopostulados/metodo5.csv')

In [13]:
test = pd.merge(noPostu,postulaciones,on=['idpostulante','idaviso'],how='inner')
len(test)

avisosNoPostuladoTodos = pd.merge(noPostu,postulaciones,on=['idpostulante','idaviso'],how='outer')
avisosNoPostulado = avisosNoPostuladoTodos[avisosNoPostuladoTodos['fechapostulacion'].isnull()]

In [15]:
avisosNoPostulado.drop(['fechapostulacion','Unnamed: 0'],axis=1,inplace=True)
len(avisosNoPostulado)

C:\Users\nicol\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


7736967

In [281]:
#avisosNoPostulado.drop(['Unnamed: 0'],axis=1,inplace=True)

In [16]:
avisosNoPostulado.head()

,idpostulante,idaviso
0,ZDPADo8,1111556097
1,MVdR6N9,1111556097
2,rmRpBzd,1111556097
3,A3DzdeG,1111556097
4,0zP1kMY,1111556097


In [189]:
##PRUEBA CON SEGUNDO GENERADOR DE DOCUMENTOS NO POSTULADOS

#avisosNoPostulado = pd.read_csv('nopostulados/noPostuladosTodos.csv')
#avisosNoPostulado['idpostulante'] = avisosNoPostulado['idpostulante_x']
#avisosNoPostulado.drop(['Unnamed: 0','idpostulante_y','fechapostulacion','idpostulante_x'],axis=1,inplace=True)



##GENERADOS CON METODO 4
#avisosNoPostulado = pd.read_csv('nopostulados/NopostuladosNuevoMetodo.csv')
#avisosNoPostulado.drop(['Unnamed: 0','fechapostulacion'],inplace=True,axis=1)


#Levanto avisos no postulados --old version
#avisosNoPostulado = pd.DataFrame();
#for i in range(0,15):
#    noPostulado = pd.read_csv('nopostulados/nopostulados-'+str(i)+'.csv')
#    avisosNoPostulado = pd.concat([noPostulado,avisosNoPostulado]);

#avisosNoPostulado.drop('Unnamed: 0',axis=1,inplace=True)
#avisosNoPostulado = avisosNoPostulado.drop_duplicates()

# Preparacion de datos

In [17]:
#Convierto en date
postulaciones['datePostulacion'] = pd.DatetimeIndex(postulaciones['fechapostulacion']).date

### Transformo las empresas a numeros

In [18]:
##Junto las 3 para tener todas las empresas mencionadas.
detTodas = pd.concat([detalleCompletas,detalleCompletasZIP])
detalleMerged = detTodas[['idaviso','denominacion_empresa']]

In [19]:
#Convertimos los nombres de las empresas a numericals
#Mergeo avisos con postulaciones
postulacionesMerged = pd.merge(postulaciones,detalleMerged,on='idaviso',how='inner');

In [20]:
postulacionesMerged['cantidadEmpresa'] = 1;
postulacionesMerged['denominacion_empresa'].fillna('S/E', inplace=True);
cantAvisosPorEmpresa = postulacionesMerged.groupby('denominacion_empresa').sum()
cantAvisosPorEmpresa = cantAvisosPorEmpresa.reset_index()
cantAvisosPorEmpresa = cantAvisosPorEmpresa[['denominacion_empresa','cantidadEmpresa']]
postulacionesMerged.drop('cantidadEmpresa',axis=1,inplace=True)

In [287]:
cantAvisosPorEmpresa.sort_values(by='cantidadEmpresa',ascending=False).head(10)

,denominacion_empresa,cantidadEmpresa
2025,Manpower,963909
2490,RANDSTAD,893729
161,Adecco -Región Office,697775
1444,Grupo Gestión,689770
1236,Farmacity,588353
259,Assistem,575936
303,BBVA Francés,549225
301,BAYTON,439058
2462,Pullmen Servicios Empresarios S.A.,396778
157,Adecco - Región NORTE & OESTE GBA,328406


###  Contamos la cantidad de postulaciones por aviso

In [21]:
#Mergeo con las postulaciones
postulacionesConEmpresas = pd.merge(postulacionesMerged,cantAvisosPorEmpresa,how='inner',on='denominacion_empresa')
#Contamos la cantidad de avisos a los que se postulo
avisosPostulados = postulaciones[['idaviso']]
avisosPostulados['cantPostulaciones'] = 1;
avisosPostuladosGP = avisosPostulados.groupby(['idaviso']).sum()
avisosPostuladosGP = avisosPostuladosGP.reset_index()

C:\Users\nicol\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [22]:
avisosPostuladosGP.sort_values(by='cantPostulaciones',ascending=False).head(10)

,idaviso,cantPostulaciones
10405,1112334791,38676
1084,1112094756,35185
931,1112033906,34738
10402,1112334788,26531
1837,1112204682,23379
11224,1112345900,22590
1640,1112196813,22109
9732,1112319451,20814
6977,1112280937,20784
8199,1112298966,19621


## Reducimos el set de postulaciones para trabajar

#### Me quedo con un subset mucho mas chico, puesto que son demasiados datos.

In [49]:
avisosPostuladosSample = postulaciones.sample(frac=0.08)
avisosPostuladosSample = avisosPostuladosSample[['idaviso','idpostulante']]
len(avisosPostuladosSample)

937031

In [50]:
avisosNOPostuladosSample = avisosNoPostulado.sample(frac=0.07)
avisosNOPostuladosSample = avisosNOPostuladosSample[['idaviso','idpostulante']]
len(avisosNOPostuladosSample)

541588

In [51]:
#Concateno los no postulados, con los postulados
avisosNOPostuladosSample['sepostulo'] = 0;
avisosPostuladosSample['sepostulo'] = 1;
datosMergeados = pd.concat([avisosNOPostuladosSample,avisosPostuladosSample])

In [52]:
len(datosMergeados)

1478619

# Convierto zona, tipo de trabajo y nivel laboral en numerical

#####  Primero uno los datasets de testing y datos de training, de forma que pueda trabajarlos en conjunto,  y no tener problemas con columnas faltantes

In [71]:
#Marco los sets de training con el de test
testingSet['testSet'] =1;
datosMergeados['testSet'] = 0;

In [72]:
#Ahora unimos los detalles con los avisos
datosMergeadosConDetalle = pd.merge(datosMergeados,detalleCompletas,how='inner',on='idaviso');
testingSetConDetalle = pd.merge(testingSet,detalleCompletasZIP,how='left',on='idaviso');

In [55]:
len(testingSetConDetalle)

100000

In [77]:
#Preparo el set de datos para unirlo y pasarlo a categorical
datosConTestSet = pd.concat([datosMergeadosConDetalle,testingSetConDetalle]);
datosConTestSetFiltrado = datosConTestSet;

In [78]:
testingSetConDetalle.columns

Index(['id', 'idaviso', 'idpostulante', 'testSet', 'idpais', 'titulo',
       'descripcion', 'nombre_zona', 'ciudad', 'mapacalle', 'tipo_de_trabajo',
       'nivel_laboral', 'nombre_area', 'denominacion_empresa'],
      dtype='object')

In [79]:
datosMergeadosConDetalle.columns

Index(['idaviso', 'idpostulante', 'sepostulo', 'testSet', 'idpais', 'titulo',
       'descripcion', 'nombre_zona', 'ciudad', 'mapacalle', 'tipo_de_trabajo',
       'nivel_laboral', 'nombre_area', 'denominacion_empresa'],
      dtype='object')

###  Convierto nombre_zona, tipo_de_trabajo y nivel_laboral a Numerical

In [80]:
datosOHC = pd.concat([datosConTestSetFiltrado,pd.get_dummies(datosConTestSetFiltrado['nombre_zona'], prefix='zona')],axis=1)
datosOHC = pd.concat([datosOHC,pd.get_dummies(datosOHC['tipo_de_trabajo'], prefix='tipo')],axis=1)
datosOHC = pd.concat([datosOHC,pd.get_dummies(datosOHC['nivel_laboral'], prefix='nivel')],axis=1)

datosOHC.drop(['nivel_laboral','tipo_de_trabajo','nombre_zona'],axis=1,inplace=True)

In [81]:
datosOHC['denominacion_empresa'].fillna('S/E', inplace=True)
datosOHC = pd.merge(datosOHC,cantAvisosPorEmpresa,how='left',on='denominacion_empresa')
datosOHC.drop('denominacion_empresa',axis=1,inplace=True);

### Preparo Genero y Estudios

In [82]:
generoTodas.head()

,idpostulante,fechanacimiento,sexo,edad
0,6MM,1985-01-01,MASC,33.0
4,ebE,1952-07-07,MASC,65.0
7,NAjM,1962-06-09,FEM,56.0
10,ZjlZ,1970-01-25,MASC,48.0
11,8wPl,1973-03-13,MASC,45.0


###  Creo una columna edad

In [83]:
import datetime as dt

generoTodas['fechanacimiento'] = pd.to_datetime(generoTodas['fechanacimiento'], format='%Y-%m-%d', errors='coerce')
generoTodas = generoTodas.dropna(subset=['fechanacimiento'])

hoy = pd.Timestamp(dt.datetime.now())
generoTodas['edad'] = (hoy - generoTodas['fechanacimiento']).astype('<m8[Y]')

generoTodas = generoTodas.loc[generoTodas['edad'].notnull(),:]
generoTodasFiltered = generoTodas.loc[((generoTodas['edad'] >= 17) & (generoTodas['edad'] < 80 )), :]

### Convierto a numerical el sexo

In [84]:

postulantesGeneroOHC = pd.concat([generoTodas,pd.get_dummies(generoTodas['sexo'], prefix='sexo')],axis=1)
postulantesGeneroOHC = postulantesGeneroOHC[postulantesGeneroOHC['sexo_0.0'] == 0]
postulantesGeneroOHC = postulantesGeneroOHC[['idpostulante','sexo_FEM','sexo_MASC','sexo_NO_DECLARA','edad']]

postulantesGeneroOHC.drop_duplicates(subset='idpostulante',inplace=True)

### Trabajo con la educacion

#### Uno las dos columnas de nombre y estado, de forma de no perder la relacion de estudio-estado

In [85]:
educacionTodas['nombreEstado'] = educacionTodas['nombre'] +" - " + educacionTodas['estado']
educacionOHC = pd.concat([educacionTodas,pd.get_dummies(educacionTodas['nombreEstado'], prefix='educacion')],axis=1)
educacionOHC.drop(['nombre','estado','nombreEstado'],axis=1,inplace=True)

In [86]:
educacionOHCGP = educacionOHC.groupby('idpostulante').sum()
educacionOHCGP = educacionOHCGP.reset_index()

In [87]:
len(educacionOHCGP)

447909

In [88]:
len(postulantesGeneroOHC)

478309

In [89]:
educacionOHCGP.head()

,idpostulante,educacion_Doctorado - Abandonado,educacion_Doctorado - En Curso,educacion_Doctorado - Graduado,educacion_Master - Abandonado,educacion_Master - En Curso,educacion_Master - Graduado,educacion_Otro - Abandonado,educacion_Otro - En Curso,educacion_Otro - Graduado,...,educacion_Posgrado - Graduado,educacion_Secundario - Abandonado,educacion_Secundario - En Curso,educacion_Secundario - Graduado,educacion_Terciario/Técnico - Abandonado,educacion_Terciario/Técnico - En Curso,educacion_Terciario/Técnico - Graduado,educacion_Universitario - Abandonado,educacion_Universitario - En Curso,educacion_Universitario - Graduado
0,0z5Dmrd,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0z5JW1r,0,0,0,0,0,0,0,0,2,...,0,0,0,2,0,0,2,2,0,0
2,0z5VvGv,0,0,0,0,0,0,0,0,0,...,0,0,0,2,0,0,0,0,0,0
3,0zB01pE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,0,0
4,0zB026d,0,0,0,0,0,0,0,0,0,...,0,0,0,2,0,0,2,0,0,0


#### Uno los generos y los estudios en un unico dataframe

In [90]:
generoYEducacionOHC  = pd.merge(postulantesGeneroOHC,educacionOHCGP,how='left',on='idpostulante')

In [91]:
len(generoYEducacionOHC)

478309

### Cuento cantidad de vistas de  avisos

In [92]:
#Set de testing
#vistasZIP['timestamp'] = pd.to_datetime(vistasZIP['timestamp'])
vistasZIP['dateVista'] = pd.DatetimeIndex(vistasZIP['timestamp']).date

In [48]:
vistasZIP['cantidadVistos'] = 1;
#vistasZIP.drop('timestamp',axis=1,inplace=True)
#vistasZIPGP = vistasZIP.groupby(['idAviso','idpostulante']).sum()
vistasZIPGP = vistasZIP.groupby(['idAviso']).sum()

vistasZIPGP = vistasZIPGP.reset_index()
vistasZIPGP['idaviso']  = vistasZIPGP['idAviso']
vistasZIPGP.drop('idAviso',axis=1,inplace=True);

In [93]:
vistasZIPGP.sort_values(by='cantidadVistos',ascending=False).head()

,cantidadVistos,idaviso
13310,71885,1806525
13015,60237,1806178
22436,43857,1112196813
21739,37255,1112094756
14163,31622,1807584


In [56]:
#Set de training

In [95]:
vistasCompletas['cantidadVistos'] =1;
#vistasGP = vistas.groupby(['idAviso','idpostulante']).sum()
vistasGP = vistasCompletas.groupby(['idaviso']).sum()

vistasGP = vistasGP.reset_index()
#vistasGP['idaviso']  = vistasGP['idAviso']
#vistasGP.drop('idAviso',axis=1,inplace=True);

In [96]:
vistasGP.sort_values(by='cantidadVistos',ascending=False).head()

,idaviso,cantidadVistos
1530,1112094756,39615
6188,1112334791,35614
1383,1112033906,27032
12658,1112431304,24916
3652,1112280937,23818


##  Convierto el nombre de  area a numerical

#### Similar a las empresas, sumo la cantidad que tengo, para representarla como la suma total 

In [97]:
#Unimos los detalles de los avisos con los avisos del set de test
detallesTodosPosta = pd.concat([detalleCompletas, detalleCompletasZIP])

In [98]:
detTodas = detallesTodosPosta[['nombre_area']];

In [99]:
detTodas['cantidadNombre'] = 1
detTodas['nombre_area'].fillna('Otros')
detalleTodasGP = detTodas.groupby('nombre_area').sum()

C:\Users\nicol\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [100]:
detalleTodasGP = detalleTodasGP.reset_index()
nombreConCantidad = detalleTodasGP[['nombre_area','cantidadNombre']]

In [101]:
nombreConCantidad.sort_values(by='cantidadNombre',ascending=False).head()

,nombre_area,cantidadNombre
185,Ventas,5721
30,Comercial,3258
2,Administración,3116
143,Producción,2830
145,Programación,1735


## Mergeamos los datos transformados

In [102]:
datosForTesting = datosOHC[datosOHC['testSet'] == 1]
datosForTraining =datosOHC[datosOHC['testSet'] == 0]

In [103]:
len(postulaciones['idpostulante'].unique())

348883

In [104]:
datosForTesting.columns.size

43

In [105]:
datosForTraining.columns.size

43

In [106]:
datosForTraining.columns

Index(['ciudad', 'descripcion', 'id', 'idaviso', 'idpais', 'idpostulante',
       'mapacalle', 'nombre_area', 'sepostulo', 'testSet', 'titulo',
       'zona_Buenos Aires (fuera de GBA)', 'zona_Capital Federal',
       'zona_Catamarca', 'zona_Ciudad de Mendoza', 'zona_Cordoba',
       'zona_Corrientes', 'zona_GBA Oeste', 'zona_Gran Buenos Aires',
       'zona_La Plata', 'zona_Mendoza', 'zona_Neuquen', 'zona_Rosario',
       'zona_San Juan', 'zona_Santa Cruz', 'zona_Santa Fe', 'zona_Tucuman',
       'tipo_Fines de Semana', 'tipo_Full-time', 'tipo_Part-time',
       'tipo_Pasantia', 'tipo_Por Contrato', 'tipo_Por Horas',
       'tipo_Primer empleo', 'tipo_Teletrabajo', 'tipo_Temporario',
       'tipo_Voluntario', 'nivel_Gerencia / Alta Gerencia / Dirección',
       'nivel_Jefe / Supervisor / Responsable', 'nivel_Junior', 'nivel_Otro',
       'nivel_Senior / Semi-Senior', 'cantidadEmpresa'],
      dtype='object')

#### Mergeo con Genero y Edad

In [107]:
len(datosForTraining)

2454734

In [108]:
datosForTrainingConGyE = pd.merge(datosForTraining,generoYEducacionOHC,on='idpostulante',how='inner')
datosForTestingConGyE = pd.merge(datosForTesting,generoYEducacionOHC,on='idpostulante',how='left')

In [109]:
len(datosForTrainingConGyE)

2394400

#####  Mergeo con nombre area

In [116]:
#Mergeamos con los nombres

datosForTrainingConGyE['nombre_area'] = datosForTrainingConGyE['nombre_area'].fillna('Otros')
datosForTestingConGyE['nombre_area'] = datosForTestingConGyE['nombre_area'].fillna('Otros')

datosForTrainingConNombre = pd.merge(datosForTrainingConGyE,nombreConCantidad,on='nombre_area',how='left')
datosForTestingConNombre = pd.merge(datosForTestingConGyE,nombreConCantidad,on='nombre_area',how='left')

datosForTrainingConNombre['cantidadNombre'] = datosForTrainingConNombre['cantidadNombre'].fillna(0)
datosForTestingConNombre['cantidadNombre'] = datosForTestingConNombre['cantidadNombre'].fillna(0)

#### Mergeamos con cantidad vistas

In [117]:
datosConVistas = pd.merge(datosForTrainingConNombre, vistasGP,on=['idaviso'],how='left')
testingConVistas = pd.merge(datosForTestingConNombre,vistasZIPGP,on=['idaviso'],how='left')

In [118]:
len(datosConVistas)

2394400

In [119]:
testingConVistas['cantidadVistos'] = testingConVistas['cantidadVistos'].fillna(0)
datosConVistas['cantidadVistos'] = datosConVistas['cantidadVistos'].fillna(0)

In [329]:
testingConVistas.columns.size

70

# Trabajo con la descripcion 

In [270]:
def limpioTextos(df):
    df['descripcionSinHTML'] = df['descripcion'].str.strip()
#elimino tags de descripcion
    df['descripcionSinHTML'] = df['descripcionSinHTML'].str.replace(r"<\/?\w+.*?>","");
    df['descripcionSinHTML'] = df['descripcionSinHTML'].str.replace(r"\s{2,}"," ");
    df['descripcionSinHTML'] = df['descripcionSinHTML'].str.lower
    return df;

In [271]:
descripcionesLimpiadas = limpioTextos(datosConVistas)
testingDescripcionesLimpiadas = limpioTextos(testingConVistas)

In [52]:
descripcionesSolas = datosConVistasLimpiado[['descripcionSinHTML']]

In [54]:
descripcionesSolas = descripcionesSolas.drop_duplicates()

In [55]:
, regex=False

# Machine Learning

In [120]:
import pandas
from sklearn import model_selection
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import RadiusNeighborsRegressor
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn import linear_model
from sklearn import svm
from sklearn import tree
from sklearn import gaussian_process
from sklearn import neural_network

#Pre-procesamiento
from sklearn import preprocessing
from sklearn.pipeline import make_pipeline

In [121]:
datosML = datosConVistas
testingML = testingConVistas

In [122]:
datosSinPostulacion = datosML.drop(['idpais','sepostulo','idaviso','idpostulante','id','testSet','nombre_area','descripcion','titulo','mapacalle','ciudad'],axis=1)
datosToPredict = testingML.drop(['idpais','sepostulo','idaviso','idpostulante','id','testSet','nombre_area','descripcion','titulo','mapacalle','ciudad'],axis=1)

In [123]:
con1 = datosML[datosML['sepostulo'] == 1]
con0 = datosML[datosML['sepostulo'] == 0]

In [124]:
len(con1)

1543693

In [125]:
len(con0)

850707

In [336]:
datosSinPostulacion.columns.size

59

In [337]:
datosToPredict.columns.size

59

In [338]:
#datosSinPostulacion.drop('fechapostulacion',axis=1,inplace=True)
#datosToPredict.drop('fechapostulacion',axis=1,inplace=True)

In [126]:
from sklearn.preprocessing import Imputer
imp = Imputer(missing_values='NaN', strategy='most_frequent', axis=0)
datosForTrainingSinPostuSinNulos = imp.fit_transform(datosSinPostulacion)
datosForTestFinalSinNulos = imp.fit_transform(datosToPredict)

In [128]:
X_train, X_validation, Y_train, Y_validation = model_selection.train_test_split(datosForTrainingSinPostuSinNulos, datosML['sepostulo'], test_size=0.7, random_state=7)

In [129]:
models = []
models.append(('RFC100',  RandomForestClassifier(n_estimators=100, max_features='sqrt')))


for name, model in models:
    m = model.fit(X_train, Y_train)
    pruebas = m.predict(datosForTestFinalSinNulos)
    archivo = pd.DataFrame()
    archivo['id'] = testingSet.loc[:,['id']]
    archivo['sepostulo'] =np.round(pruebas)
    archivo.to_csv('entregas/archivolimpio/'+'prueba4-archivosFaltantes'+ name +'.csv', index=False)

In [130]:
from sklearn.model_selection import cross_val_score

RFCModel = RandomForestClassifier(n_estimators=100, max_features='sqrt')
scoresRFC = cross_val_score(RFCModel, datosForTrainingSinPostuSinNulos, datosML['sepostulo'], cv=5)
print("---RFC----")
print(scoresRFC)


---RFC----
[ 0.75998212  0.75810274  0.77327723  0.76946995  0.75825835]
